# Convolution on GPU

Modern machine learning is deeply intertwined with the concept of GPU computations. However, a GPU is not a magic button that automatically speeds up every operation. To fully harness the benefits of modern GPUs, algorithms must be carefully designed to run smoothly on these parallel architectures, leveraging concepts such as SIMD (Single Instruction, Multiple Data) along with other optimization techniques.

In this notebook, I will dive into the details of how to implement a convolution operation an essential component of convolution layers—specifically tailored to work efficiently on the GPU.

This work will include theory, code and evaluation of the algorithm. For the implementation I will use numpy and cupy as the main tool, and also I will research on the numba tool and evaluate the ways to optimize the algorithm even further.

In [1]:
import numpy as np
import cupy as cp
from numba import njit

## Basic Convolution Algorithm

In the introductory lectures to the Convolutional Neural Networks, the most basic algorithm is introduced: it slides over each patch along the $i$ and $j$ axes and accumulates the result of the matrix multiplication between the input patch and the convolution kernel. In mathematical terms, for a single chanel of input $I$ and kernel $K$, the output at position $(i,j)$ is computed as:

$$
O(i,j) = \sum_{m=0}^{k_h-1} \sum_{n=0}^{k_w-1} I(i:i+m, j:j+n) \cdot K(m,n)
$$

In [2]:
def forward(self, X):
    # get shape of the input tensor
    batch_size, height, width, in_channels = X.shape
    # compute shape of output tensor
    out_height = height - self.filter_size + 1 + 2 * self.padding
    out_width = width - self.filter_size + 1 + 2 * self.padding
    # save last input for backward pass
    self.last_X = X
    # create zeros tensor for result
    result = np.zeros((batch_size, out_height, out_width, self.out_channels))
    # reshape weights to use matrix multiplication trick
    weights = self.W.value.reshape(self.in_channels * self.filter_size * self.filter_size, self.out_channels)
    # iterate each pixel in output tensor
    for y in range(out_height):
        for x in range(out_width):
            # take the perception widow of the output pixel
            patch = X[:, y:y + self.filter_size, x:x + self.filter_size, :]
            # unwrap patch to use matrix multiplication trick
            patch_flat = patch.reshape(batch_size, self.in_channels * self.filter_size * self.filter_size)
            # convolution operation
            res = patch_flat @ weights + self.B.value
            # add pixels to result tensor
            result[:, y, x, :] = res
    return result

While this approach is conceptually simple and useful to understand the idea of convolution, it has several limitations:

1. **Inefficient Looping Structure:**
   The basic algorithm uses nested loops to traverse the input chanel, resulting in $O(H \times W \times k_h \times k_w)$ time complexity. This becomes computationally expensive as the input size or the kernel dimensions increase.

2. **Poor Memory Access Patterns:**
   Iterating patch-by-patch can lead to inefficient memory access, as data may not be loaded into the cache optimally. This results in significant overhead, especially when running on architectures where memory bandwidth is a critical bottleneck.

3. **Limited Parallelism:**
   GPUs thrive on parallel processing, but the naive implementation with its serial loop structure fails to exploit the inherent parallelism. The lack of vectorized operations means that the GPU's SIMD (Single Instruction, Multiple Data) capabilities remain underutilized.

These challenges motivate the need for optimized approaches that reorganize computations and memory access patterns, allowing us to fully harness the power of GPU acceleration for convolution operations.

---
## Convolution on GPU Algorithm

To start developing the algorithm, it is essential to clearly structure and understand all the input data:

- **Input Tensor ($I$)**: with shape $(batch\_size \times filter\_size \times filter\_size \times in\_channels)$.
- **Weights Tensor ($W$)**: with shape $(filter\_size \times filter\_size \times in\_channels \times out\_channels)$.
- **Bias Tensor ($b$)**: with shape $(filter\_size \times filter\_size \times in\_channels \times out\_channels)$.

In introductory lectures on convolution operations, the most basic algorithm slides a kernel over each patch along the $i$ and $j$ axes, performing an element-wise multiplication of the patch and the weights, then summing the result to produce the output. The core issue with this basic approach is that each convolution operation is executed separately on small data patches (of size $filter\_size \times filter\_size$) using implicit iteration loops.

Our goal is to transform these isolated operations into a vectorized form. The approach taken here represents the entire convolution over the whole image as a single matrix multiplication. To achieve this, I introduce the `im2conv_matrix` operation, which effectively reshapes and aggregates the convolution patches into a form amenable to efficient, parallel computation on the GPU.

---
### im2conv_matrix

The idea behind this operation is to transform the input tensor into a single matrix with the following shape:
$$
(batch\_size \cdot H\_out \cdot W\_out \times filter \cdot filter \cdot in\_channels)
$$

where:
- $W\_out$ represents the total number of patches along the width.
- $H\_out$ represents the total number of patches along the height.

In this configuration, each row corresponds to a flattened patch extracted from the input tensor, encompassing all channels.

Next, we reshape the weights tensor to have the shape:
$$
(out\_channels \times filter \cdot filter \cdot in\_channels)
$$
This reshaping organizes the filters such that each row now represents a complete set of filter weights for all input channels corresponding to a specific output channel. With this setup, we can perform the convolution operation as a single matrix multiplication, which is highly efficient on the GPU.

Then the result of Convolution operation could be done as:
$$
\begin{aligned}
&result = input\_reshaped \cdot weights^T \\
&\text{with shape } result (batch\_size \cdot H\_out \cdot W\_out \times out\_channels)
\end{aligned}
$$

---
### im2conv_matrix logic

Suppose input is tensor, without batching with 2 channels, then the result of the `im2conv_matrix` operation should look like:

<img src="imgs/conv_matrix.jpg">

so every row in the `im2conv_matrix` is single conv operation.

The approach to achieve that structured below:

#### Approach:

1. Create arrays representing starting indices of each patch along $x$ and $y$ axes:

```python
x_start = np.arange(0, h - filter, stride)
y_start = np.arange(0, w - filter, stride)  # similarly for y-axis
```

---

2. Next, from these starting indices along each axis, we create a mesh-grid matrix. This allows us to identify the starting indices of each patch along both height and width dimensions for every output position:

```text
y_starts, x_starts = np.meshgrid(x_start, y_start, indexing='ij')
y_starts: [H_out, W_out]
x_starts: [H_out, W_out]
```

---

3. After that, we need to add the ability to traverse each patch. Doing so allows us to get indices of each element within each patch.

```python
dy, dx = np.mgrid[0:filter, 0:filter]

# indices of each element within the patch along two axes:
y_indices = y_starts[:, :, None, None] + dy
x_indices = x_starts[:, :, None, None] + dx
# Resulting shape: [H_out, W_out, filter, filter]

# This matrix stores (i, j) indices for each element in every convolutional patch.
```

4. Using the results from step 3, we now have the ability to get indices of any input element. The remaining step is straightforward:

We want to use advanced indexing to transform the input $X$ into our desired format.

```python
batch_idx = np.arange(batch)[:, None, None, None]
channel_idx = np.arange(channels)[None, None, None, :]

# reshape is done so broadcasting correctly aligns the channels.
```

---

5. We need to construct a matrix by sequential indexing of the input tensor $X$, such that for each:

- Batch dimension,
- Channel dimension,
- Patch starting point along $x$,
- Patch starting point along $y$,
- And every possible position within a patch,

the corresponding value from the input $X$ is stored:

The resulting shape is:

$$
(batch\_size, H\_out, W\_out, filter, filter, channels)
$$

Implemented as:

```python
res = X[batch_idx,
        y_indices,  # stores y indices for all positions
        x_indices,  # stores x indices for all positions
        channel_idx]
```

---

6. Last step: reshape into the desired format:

$$
(batch\_size \cdot H\_out \cdot W\_out, filter, filter, channels)
$$

---

Okay, once this is completed, the convolution operation becomes simple:

- **Weights tensor shape:**

$$
(in\_channels \cdot filter \cdot filter, out\_channels)
$$

- **im2col matrix shape:**

$$
(batch\_size \cdot H\_out \cdot W\_out, in\_channels \cdot filter \cdot filter)
$$

The result (`res`) is computed and bias (`B`) is added:

$$
res + B \quad \text{shape: }(batch\_size, H\_out \cdot W\_out, out\_channels)
$$

Finally, reshape the result back to:

$$
(batch\_size, H\_out, W\_out, out\_channels)
$$

This completes the forward pass.



In [3]:
def im2col_idx(x_shape, filter_size, stride):
    """
    Generate indices required for extracting image patches for convolution operation.

    Params:
        x_shape: Tuple representing the shape of input tensor (batch_size, height, width, channels)
        filter_size: Size of convolutional filter (assumed square)
        stride: Stride size for sliding the filter

    Returns:
        batch_indices: Indices corresponding to batch dimension
        y_indices: Indices for each patch along height dimension
        x_indices: Indices for each patch along width dimension
        channel_indices: Indices corresponding to channel dimension
    """
    batch_size, h, w, ch = x_shape

    # Corrected: y_start for height, x_start for width
    y_start = np.arange(0, h - filter_size + 1, stride)
    x_start = np.arange(0, w - filter_size + 1, stride)

    # Corrected: meshgrid(y_start, x_start)
    y_starts, x_starts = np.meshgrid(y_start, x_start, indexing='ij')  # [H_out, W_out]

    dy, dx = np.mgrid[0:filter_size, 0:filter_size]  # [filter_size, filter_size]

    y_indices = y_starts[:, :, None, None] + dy  # [H_out, W_out, k, k]
    x_indices = x_starts[:, :, None, None] + dx  # [H_out, W_out, k, k]

    # Add batch and channel dimensions
    batch_indices = np.arange(batch_size)[:, None, None, None, None, None]  # [N, 1, 1, 1, 1, 1]
    channel_indices = np.arange(ch)[None, None, None, None, None, :]  # [1, 1, 1, 1, 1, C]

    return batch_indices, y_indices, x_indices, channel_indices


def im2col_matrix(X, filter_size, stride):
    """
    Converts batched images into a matrix format suitable for efficient GPU-based convolution operations.

    Params:
        X: Input tensor of images with shape (batch_size, height, width, in_channels)
        filter_size: Size of convolutional filter (assumed square)
        stride: Stride size for sliding the convolutional filter

    Returns:
        A reshaped matrix suitable for convolution with shape:
        (batch_size * height_out * width_out, filter_size * filter_size * in_channels)
    """
    batch_size, h, w, ch = X.shape
    h_out = (h - filter_size) // stride + 1
    w_out = (w - filter_size) // stride + 1

    batch_indices, y_indices, x_indices, channel_indices = im2col_idx(X.shape, filter_size, stride)

    # Expand indices for broadcasting
    y_indices_exp = y_indices[np.newaxis, :, :, :, :, np.newaxis]  # [1, H_out, W_out, k, k, 1]
    x_indices_exp = x_indices[np.newaxis, :, :, :, :, np.newaxis]  # [1, H_out, W_out, k, k, 1]

    # Extract patches
    patches = X[
        batch_indices,  # [N, 1, 1, 1, 1, 1]
        y_indices_exp,  # [1, H_out, W_out, k, k, 1]
        x_indices_exp,  # [1, H_out, W_out, k, k, 1]
        channel_indices # [1, 1, 1, 1, 1, C]
    ]  # [N, H_out, W_out, k, k, C]

    # Reshape to [N * H_out * W_out, k * k * C]
    return patches.reshape(batch_size * h_out * w_out, -1)

In [4]:
a = np.arange(9).reshape((1, 3, 3, 1))

<img src="imgs/input1.png" width="600px">

In [6]:
res1 = im2col_matrix(a, filter_size=2, stride=1)

<img src="imgs/output1.png" width="600px">

Here every row is one convolutional kernel, now lets check the batching and channels.

In [19]:
a = np.arange(36).reshape((2, 3, 3, 2))

In [20]:
images = a  # Shape: (2, 2, 3, 3)

# Now, images[batch, channel] gives you the 3x3 image.
print("Image for batch 0, channel 0:")
print(images[0, 0])
print("Image for batch 0, channel 1:")
print(images[0, 1])
print("Image for batch 1, channel 0:")
print(images[1, 0])
print("Image for batch 1, channel 1:")
print(images[1, 1])

Image for batch 0, channel 0:
[[0 1]
 [2 3]
 [4 5]]
Image for batch 0, channel 1:
[[ 6  7]
 [ 8  9]
 [10 11]]
Image for batch 1, channel 0:
[[18 19]
 [20 21]
 [22 23]]
Image for batch 1, channel 1:
[[24 25]
 [26 27]
 [28 29]]


| Batch 0 Channel 0                           | Batch 0 Channel 1                           |
|---------------------------------------------|---------------------------------------------|
| <img src="imgs/input2.1.png" width="400px"> | <img src="imgs/input2.2.png" width="400px"> |

| Batch 1 Channel 0                           | Batch 1 Channel 1                           |
|---------------------------------------------|---------------------------------------------|
| <img src="imgs/input2.3.png" width="400px"> | <img src="imgs/input2.4.png" width="400px"> |


In [16]:
res2 = im2col_matrix(a, filter_size=2, stride=1)

print(res2)

[[ 0  9  1 10  3 12  4 13]
 [ 1 10  2 11  4 13  5 14]
 [ 3 12  4 13  6 15  7 16]
 [ 4 13  5 14  7 16  8 17]
 [18 27 19 28 21 30 22 31]
 [19 28 20 29 22 31 23 32]
 [21 30 22 31 24 33 25 34]
 [22 31 23 32 25 34 26 35]]


<img src="imgs/output2.png" width="600px">

In [15]:
inp[0, 0, 1, 0]

3